In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory


In [ ]:
IMG_SIZE = (224,224)
IMAGE_PATH = os.path.join(os.getcwd(), 'images')
TOTAL_NUMBER_OF_IMAGES = len(os.listdir(IMAGE_PATH))


In [ ]:
BREED_DICT = { # 0 for cat, 1 for dog
    'persian': 0,
    'ragdoll': 0,
    'havanese': 1,
    'sphynx': 0,
    'keeshond': 1,
    'beagle': 1,
    'leonberger': 1,
    'english_cocker_spaniel': 1,
    'german_shorthaired': 1,
    'saint_bernard': 1,
    'japanese_chin': 1,
    'pug': 1,
    'newfoundland': 1,
    'yorkshire_terrier': 1,
    'basset_hound': 1,
    'birman': 0,
    'great_pyrenees': 1,
    'boxer': 1,
    'staffordshire_bull_terrier': 1,
    'wheaten_terrier': 1,
    'egyptian_mau': 0,
    'maine_coon': 0,
    'american_pit_bull_terrier': 1,
    'shiba_inu': 1,
    'miniature_pinscher': 1,
    'samoyed': 1,
    'abyssinian': 0,
    'pomeranian': 1,
    'russian_blue': 0,
    'bombay': 0,
    'english_setter': 1,
    'chihuahua': 1,
    'british_shorthair': 0,
    'siamese': 0,
    'bengal': 0,
    'american_bulldog': 1,
    'scottish_terrier': 1
}



In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os


from torchvision.transforms import v2 as T
from tqdm import tqdm

from PIL import Image

import matplotlib.pyplot as plt

def get_breed(image_path):
    name = image_path.split('.')[0]
    # Split on last _
    name = '_'.join(name.rsplit('_', -1)[:-1]).lower()

    return name

def load_classes(dataset_path):
    classes = set()

    for img in os.listdir(dataset_path):
        name = get_breed(img)
        classes.add(name)

    return classes

classes = load_classes(IMAGE_PATH)
print(f"{len(classes)} classes found")

def imshow(path: torch.Tensor):
    img = path.permute(1, 2, 0)
    plt.imshow(img)
    plt.show()


class CatAndDogsDataset(Dataset):
    def __init__(self, image_path, img_size, transform=None):
        self.image_path = image_path
        self.img_size = img_size
        self.transform = transform
        self.image_files = os.listdir(self.image_path)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_path, self.image_files[idx])
        img = Image.open(img_path).convert("RGB")
        img = transforms.Resize(self.img_size)(img)

        if self.transform:
            img = self.transform(img)

        one_hot = np.zeros(2)
        label = BREED_DICT[get_breed(self.image_files[idx])]
        one_hot[label] = 1

        return img, one_hot

In [ ]:
dataset = CatAndDogsDataset(IMAGE_PATH, IMG_SIZE, transform=transforms.ToTensor())
# Take only 128  images
dataset.image_files = dataset.image_files[:128]

# Get some data
random_index = np.random.randint(0, len(dataset))
data, one_hot = dataset[random_index]

print(f"ITS A {'dog' if one_hot[1] == 1 else 'cat'}")

imshow(data)

In [ ]:
from tqdm import tqdm
from torch.utils.data import random_split


model = torchvision.models.resnet18(weights="DEFAULT")

num_classes = 2  # 2 classes: cat and dogs

in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, num_classes)

loss_criter = nn.CrossEntropyLoss()

# Use nADAM
optimizer = optim.Adam(model.parameters(), lr=0.001)

BATCH_SIZE = 64


val_size = int(0.2 * len(dataset))  # 20% for validation
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 80% of the data is used for training, 20% for validation
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

loss_over_time = []
correct = 0

for epoch in range(2):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(train_data, 0)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = loss_criter(outputs, labels)

        # Check if the model predicted the right class
        _, predicted = torch.max(outputs, 1)
        labels = torch.argmax(labels, 1)  # convert from one-hot encoding to class labels
        correct += (predicted == labels).sum().item()

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loss_over_time.append(loss.item())
        if i % 2000 == 1999:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 2000}")
            running_loss = 0.0

print(f"Accuracy: {correct / 256}")
print(running_loss)
plt.plot(loss_over_time)
print("Finished Training")